In [1]:
import pandas as pd
import bs4 as bs
import validar as val
import reqhtml as rh
import functionsjv as jv
import requests
import sqlite3

### Accesando os APIs/Web

* URLs para conectar APIS/Web


In [2]:
aeroportos = "https://pt.wikipedia.org/wiki/Lista_de_aeroportos_do_Brasil_por_c%C3%B3digo_aeroportu%C3%A1rio_ICAO"

cptec = "https://brasilapi.com.br/api/cptec/v1/clima/capital"

ibge_estados = "https://brasilapi.com.br/api/ibge/uf/v1/{code}"



#### Validando a conexao das API e da URL da tabela HTML

In [3]:
list = []
bases = []

val.conexao(aeroportos, list, bases = 0)   #Conexao da base com a tabela de aeroportos, se usa parametro bases = 0 porque esta base retorna um HTML

val.conexao(cptec, list, bases) #Conexao com a base API de Centro de Previsao de Tempo e Estudos Climaticos (cptec)

val.conexao(ibge_estados.replace('{code}', 'SP'), list, bases) #Conexao base ibge_estados, esta base precisa de um code que seria a sigla UF. 
                                                        # se faz um replace de {code} por SP para testar conexao

[{'https://pt.wikipedia.org/wiki/Lista_de_aeroportos_do_Brasil_por_c%C3%B3digo_aeroportu%C3%A1rio_ICAO': 200},
 {'https://brasilapi.com.br/api/cptec/v1/clima/capital': 200},
 {'https://brasilapi.com.br/api/ibge/uf/v1/SP': 200}]

#### Extraindo dados e criando os dataframes

* Extraindo dados para base ibge_estados para todas unidaded as federativas

In [4]:
# A base "https://brasilapi.com.br/api/ibge/uf/v1/{code}" precisa de um code para trazer os dados, esse code é a sigla da unidade federativa(UF)

#Se cria uma lista com todas a siglas federativas para poder remplazar o code da URL e criar um dataframe com todos os estado.

uf = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI','CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG',
             'ES', 'RJ', 'SP', 'PR', 'SC', 'RS', 'MS', 'MT', 'GO', 'DF']


lista = []  # Lista vazia para armazenar os valores 
 
# Se usa um For para passar cada sigla UF na função retorna_estado, que adiciona cada resultado numa lista que é retornada para se armazanada na lista vazia criada.
for sigla in uf:
    jv.retorna_estado(sigla, lista)

#lista   # para testar se os dados foram armazenados.

* Com os dados extraidos, se criam os data frames para cada base

In [5]:
#Se criam os DataFrames das bases extraídas dos APIs e do sitio Web

df_aeroporto = rh.basehtml(aeroportos) #Função rh.basehtml conecta com a url, extrai a tabela do HTML, extrai os dados e retorna um DataFrame         
df_cptec = pd.DataFrame(bases[0]) 
df_estados = pd.DataFrame(lista).reset_index(drop=True)


# Tratamento de dados

* Formatação de dataframes
* Eliminaçõ de valores null
* Mudança do tipo de dado da coluna
* Remover duplicatas

### Dataframe Aeroporto

In [6]:
#Informação da base aeroportos
df_aeroporto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nome usual    78 non-null     object
 1   nome oficial  78 non-null     object
 2   codigo        78 non-null     object
 3   localizacao   78 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


* Não tem valores nulos, então revisamos as colunas

In [7]:
df_aeroporto.tail(5)

,nome usual,nome oficial,codigo,localizacao
73,Tefé,Aeroporto Regional de Tefé,SBTF,Tefé/AM
74,Teresina\n,Aeroporto de Teresina\n,SBTE\n,Teresina/PI\n
75,Óbidos Tirios,Base Área de Tirios,SBTS,Óbidos/PA
76,Unaí,Aeroporto de Unaí,SNUN,Unaí/MG
77,Xapuri,Aeroporto de Xapuri,SWXU,Xapuri/AC


* Na coluna localizacao a cidade e as siglas UF estão juntas, vamos separar as siglas e colocar numa coluna nova.
* Tem valores com um \n que deve ser eliminado. Exemplo a linha 74


In [8]:
# Se cria uma lista vazia para armazenar cada tratamento feito na base e depois imprimir
lista_tratamento = []

#se chama a função tratamento.aeroporto que está no modulo importado functionsjv com alias jv, 
#aí se realiza a separação da coluna localização em duas novas colunas, cidade e uf. depois se remove a coluna localizacao.
# A funcão tratamento_aeroporto conta com o código para remover o caráter \n

df_aeroporto_tratada = jv.tratamento_aeroporto(df_aeroporto, lista_tratamento)
lista_tratamento

['Tratamento feito: Cidade e UF separados em duas colunas',
 'Tratamento feito: Eliminar /n dos dados',
 'Tratamento feito: Coluna localicao removida pra deixar as colunas cidade e uf']

#### Base aeroporto tratada

In [26]:
df_aeroporto_tratada

,nome usual,nome oficial,codigo,cidade,uf
0,Amapá,Aeroporto de Amapá,SBAM,Amapá,AP
1,Apuí,Aeroporto de Apuí,SWYN,Apuí,AM
2,Arapiraca,Aeroporto de Arapiraca,SNAL,Arapiraca,AL
3,Barcelos,Aeroporto de Barcelos,SWBC,Barcelos,AM
4,Barreirinha,Aeroporto de Barreirinha,SWBI,Barreirinha,AM
...,...,...,...,...,...
73,Tefé,Aeroporto Regional de Tefé,SBTF,Tefé,AM
74,Teresina,Aeroporto de Teresina,SBTE,Teresina,PI
75,Óbidos Tirios,Base Área de Tirios,SBTS,Óbidos,PA
76,Unaí,Aeroporto de Unaí,SNUN,Unaí,MG


### Dataframe CPTEC

In [27]:
#Informação da base cptec
df_cptec.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   umidade              26 non-null     int64 
 1   intensidade          26 non-null     object
 2   codigo_icao          26 non-null     object
 3   pressao_atmosferica  26 non-null     int64 
 4   vento                26 non-null     int64 
 5   direcao_vento        26 non-null     int64 
 6   condicao             26 non-null     object
 7   condicao_desc        26 non-null     object
 8   temp                 26 non-null     int64 
 9   atualizado_em        26 non-null     object
dtypes: int64(5), object(5)
memory usage: 2.2+ KB


#### Tratamento da base CPTEC
* Mudar para Timedate o tipo de dado da coluna "atualizado_em"
* Eliminar a coluna condicao porque é desnecessaria no nosso modelo, e se tem a coluna condicao_desc que explica melhor o valor.

In [9]:
#Lista vazia para armazenar o resultado da função tratamento_cptec, que retorna uma lista com os tratamentos feitos.
lista_tratamentos_cptec = []

#Se chama a função tratamento_cptec que tem o código para mudar o tipo de dado de atualizado_em de Object para Datetime,
#também contém o código que elimina com um drop() a coluna condicao.
df_cptec_tratada = jv.tratamento_cptec(df_cptec, lista_tratamentos_cptec)

lista_tratamentos_cptec  #Para imprimir os tratamentos feitos

['Tratamento feito: Valores da coluna "atualizado em" mudados para tipo Datetime',
 'Tratamento feito: Coluna "condicao" eliminada do dataframe']

* Se revisa que a base tinha mudado o tipo de dado da coluna "atualizada_em" e que a coluna "condicao" tinha sido eliminada

In [11]:
df_cptec_tratada.info()    # Se imprime para verificar que o tratamento desejado na base foi feito.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   umidade              26 non-null     int64              
 1   intensidade          26 non-null     object             
 2   codigo_icao          26 non-null     object             
 3   pressao_atmosferica  26 non-null     int64              
 4   vento                26 non-null     int64              
 5   direcao_vento        26 non-null     int64              
 6   condicao_desc        26 non-null     object             
 7   temp                 26 non-null     int64              
 8   atualizado_em        26 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5), object(3)
memory usage: 2.0+ KB


#### Base CPTEC tratada

In [12]:
df_cptec_tratada.tail(10)  #Se imprime o dataframe para revisar os dadods e colunas

,umidade,intensidade,codigo_icao,pressao_atmosferica,vento,direcao_vento,condicao_desc,temp,atualizado_em
16,83,>10000,SBPA,1015,22,100,Predomínio de Sol,25,2024-03-01 22:00:00.024000+00:00
17,89,>10000,SBPV,1011,4,310,Predomínio de Sol,27,2024-03-01 22:00:00.024000+00:00
18,79,>10000,SBRF,1012,14,30,Predomínio de Sol,29,2024-03-01 22:00:00.024000+00:00
19,94,>10000,SBRB,1012,7,270,Predomínio de Sol,26,2024-03-01 22:00:00.024000+00:00
20,79,>10000,SBRJ,1013,14,170,Predomínio de Sol,26,2024-03-01 22:00:00.024000+00:00
21,84,>10000,SBSV,1011,18,60,Predomínio de Sol,29,2024-03-01 22:00:00.024000+00:00
22,89,>10000,SBSL,1012,14,20,Predomínio de Sol,27,2024-03-01 22:00:00.024000+00:00
23,83,>10000,SBSP,1019,11,150,Predomínio de Sol,20,2024-03-01 22:00:00.024000+00:00
24,89,>10000,SBTE,1012,11,340,Predomínio de Sol,26,2024-03-01 22:00:00.024000+00:00
25,100,>10000,SBVT,1011,0,9999,Predomínio de Sol,23,2024-03-01 22:00:00.024000+00:00


### Dataframe Estados

In [13]:
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      27 non-null     int64 
 1   sigla   27 non-null     object
 2   nome    27 non-null     object
 3   regiao  27 non-null     object
dtypes: int64(1), object(3)
memory usage: 996.0+ bytes


#### Tratamento da base Estados

*Não tem valores nulos nem tipos de dados para mudar.

In [14]:
#Se revisa o Dataframe para revisar os dados
df_estados.head(5)

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


* "A coluna 'regiao' contém um dicionário de dados, e nos interessa extrair o nome da região para uma nova coluna. Este procedimento será realizado a seguir."

In [15]:
lista_tratamento_estados = []

#Se adiciona a nova coluna "nome_regiao" no dataframe df_estados.
df_estados_tratada = jv.tratamento_estados(df_estados, lista_tratamento_estados)

lista_tratamento_estados

['Tratamento feito: nome da regiao extraído e armazenado na variável nome_regiao',
 'Tratamento feito: Se adicionou a coluna "nome_regiao" no dataframe df_estados']

In [16]:
df_estados_tratada.tail(5)

,id,sigla,nome,regiao,nome_regiao
22,43,RS,Rio Grande do Sul,"{'id': 4, 'sigla': 'S', 'nome': 'Sul'}",Sul
23,50,MS,Mato Grosso do Sul,"{'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}",Centro-Oeste
24,51,MT,Mato Grosso,"{'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}",Centro-Oeste
25,52,GO,Goiás,"{'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}",Centro-Oeste
26,53,DF,Distrito Federal,"{'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}",Centro-Oeste


# Salvar Bases tratadas no formato CSV

In [17]:
lista_bases_tratadas = [df_aeroporto_tratada, df_cptec_tratada, df_estados_tratada] #Lista com os 3 dataframes obtidos
lista_nomes = ['Base Aeroporto', 'Base CPTEC', 'Base Estados'] #Lista com os nomes para cada base

jv.salvar_em_csv(lista_bases_tratadas, lista_nomes)  #Função em functionsjv.py que recebe a lista de dataframes e a lista de nomes para ser iteradas e salvar os 3 dataframes no formato .csv


'Bases salvadas no formato .csv em C:\\Temp'

# Criação de banco de dados com as bases geradas

* Se conecta ao banco ou se cria um novo banco
* Se criam as tabelas do banco de dados com as bases .CSV

In [18]:
#Se criam 2 listas, uma com as bases.csv e outra que contém os nomes para as tabelas.
lista_basecsv = ["Base Aeroporto.csv", "Base CPTEC.csv", "Base Estados.csv"] 
lista_nome_tabela = ['Aeroportos', 'Previsoes', 'Estados']

conn = sqlite3.connect('C:\\Temp\\cptec_aeroportos.db') #conectar ou criar uma db se nao existe

jv.bd_tables(lista_basecsv, lista_nome_tabela, conn) #Função para criar tabelas



"As tabelas ['Aeroportos', 'Previsoes', 'Estados'] foram criadas"

* Se verifcam que as tabelas foram criadas no banco de dados

In [19]:
#Se hace um query que retorne as tabelas do banco de dados cptec_aeroportos.db para conferir que foram criadas
query2 = "select name from sqlite_master where type='table'"
schema = pd.read_sql_query(query2, conn) # so para consultas
schema

,name
0,Aeroportos
1,Previsoes
2,Estados


* Se faz um consulta com as tabelas criadas para verificar os dados

In [20]:
#executar uma consulta que une com um JOIN dados das tabelas Previsoes e Aeroportos, filtrando valores nulos
query = '''SELECT cidade, "nome oficial", condicao_desc, codigo, codigo_icao, atualizado_em
           FROM Previsoes
           Left JOIN Aeroportos ON Previsoes.codigo_icao = Aeroportos.codigo
           WHERE cidade IS NOT NULL''' #
df = pd.read_sql(query, conn)  
df

,cidade,nome oficial,condicao_desc,codigo,codigo_icao,atualizado_em
0,Belém,Aeroporto Internacional de Belém Val-de-Cães,Predomínio de Sol,SBBE,SBBE,2024-03-01 22:00:00.024000+00:00
1,Confins,Aeroporto Internacional de Belo Horizonte-Confins,Predomínio de Sol,SBCF,SBCF,2024-03-01 22:00:00.024000+00:00
2,Lago Sul,Aeroporto Internacional de Brasília,Predomínio de Sol,SBBR,SBBR,2024-03-01 22:00:00.024000+00:00
3,São José dos Pinhais,Aeroporto Internacional Afonso Pena,Predomínio de Sol,SBCT,SBCT,2024-03-01 22:00:00.024000+00:00
4,Florianópilis,Aeroporto Internacional Hercílio Luz,Predomínio de Sol,SBFL,SBFL,2024-03-01 22:00:00.024000+00:00
5,Fortaleza,Aeroporto Internacional Pinto Martins,Predomínio de Sol,SBFZ,SBFZ,2024-03-01 22:00:00.024000+00:00
6,Goiânia,Aeroporto Internacional Santa Genoveva,Predomínio de Sol,SBGO,SBGO,2024-03-01 22:00:00.024000+00:00
7,Bayeux,Aeroporto Internacional de João Pessoa-Bayeux,Predomínio de Sol,SBJP,SBJP,2024-03-01 22:00:00.024000+00:00
8,Macapá,Aeroporto Internacional de Macapá-Alberto Alco...,Predomínio de Sol,SBMQ,SBMQ,2024-03-01 22:00:00.024000+00:00
9,Rio Largo,Aeroporto Internacional Zumbi dos Palmares,Predomínio de Sol,SBMO,SBMO,2024-03-01 22:00:00.024000+00:00
